RAG Chatbot - Core component

## Flow process: User Query → Retriever → Relevant Chunks → LLM → Final Answer

Install dependencies

In [ ]:
!pip install -qU langchain langchain-community langchain-cohere langchain-pinecone langchain-google-genai google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.9/438.9 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.0/69.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.3/524.3 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 75.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB

In [ ]:
!pip install --upgrade langchain-pinecone

Import library

In [ ]:
from langchain.vectorstores import Pinecone as LangChainPinecone
from pinecone import Pinecone, ServerlessSpec
import os
import getpass

Setup Pinecone connection

In [ ]:
# Pinecone AUTH
api_key_pinecone = getpass.getpass("Input API KEY PINECONE CLOUD")
print('api_key_pinecone telah diinput')

In [ ]:
# Initialize Pinecone
pinecone_api_key = "pcsk_4Jb3SW_97kXFsvEF55ttZuxSDPrrXzus219H3u25ZYeevkVqmXa1sRrkTAn8C7KDYfNgb3"
pc = pinecone.Pinecone(api_key=pinecone_api_key)

index_name = "nutrition-rag-index"

spec = ServerlessSpec(
    cloud="aws", region="us-east-1"
)

# index_name = 'nutrition-rag-index'
existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

# Create vector index if not exist
if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=768,
        metric='cosine',
        spec=spec
    )
    while not pc.describe_index(index_name).status['ready']:
        time.sleep(1)

# Connect to index
index = pc.Index(index_name)

Load Gemini AI as the LLM

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [ ]:
# Gemini AI AUTH
import getpass
import os

if not os.getenv("GOOGLE_API_KEY"):
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google API key: ")

In [ ]:
embedding_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

### Type-1: Connect to Pinecone Vector Store + asking basic question

In [ ]:
# Load Pinecone index
vectorstore = Pinecone.from_existing_index(index_name="nutrition-rag-index", embedding=embedding)

# Create retriever
retriever = vectorstore.as_retriever(search_type="similarity", k=5)

#### Connect Retriever + Gemini LLM

In [ ]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True,
    chain_type="stuff"
)

#### Ask Questions Using Gemini + Context

In [ ]:
query = "Suggest a low-fat recipe with broccoli and rice."
response = qa_chain.run(query)

print("Gemini Answer:\n", response)

### Type-2: Add smart filtering, personalization, and goal-based nutrition logic.

##### File: RAG_chain

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.embeddings import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain.chains import LLMChain, RetrievalQA
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from pinecone import Pinecone, ServerlessSpec
import os
import getpass

In [ ]:
# Load Pinecone index
vectorstore = Pinecone.from_existing_index(index_name="nutrition-rag-index", embedding=embedding)

retriever = vectorstore.as_retriever(search_type="similarity", k=4)

In [ ]:
# Prompt template with dietary filtering logic
prompt_template = PromptTemplate(
    input_variables=["diet", "goal", "calorie_limit", "query"],
    template="""
You are NutriBot, a smart nutrition assistant.

User's dietary profile:
- Diet type: {diet}
- Health goal: {goal}
- Max calories per meal: {calorie_limit}

Please answer the user's query in a helpful, friendly, and informative tone.

Query: "{query}"

Return only relevant meals or recipes that respect the user's dietary needs and goals.
"""
)

In [ ]:
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash",
                             temperature=0,
                             max_tokens=None,
                             timeout=None,
                             temperature=0.3)

In [ ]:
# Build LLM chain
llm_chain = LLMChain(llm=llm, prompt=prompt_template)
stuff_chain = StuffDocumentsChain(
    llm_chain=llm_chain,
    document_variable_name="context"
)

In [ ]:
# Final dietary-aware RAG QA chain
dietary_qa_chain = RetrievalQA(
    retriever=retriever,
    combine_documents_chain=stuff_chain,
    return_source_documents=True
)

##### File: Streamlit UI - main.py

In [ ]:
import streamlit as st
from chains.rag_chain import dietary_qa_chain

In [ ]:
st.set_page_config(page_title="NutriBot - Nutrition Chat", page_icon="🥦")
st.title("🥦 NutriBot - Personalized Nutrition Chatbot")

st.markdown("Ask about food, recipes, or get suggestions based on your dietary goals!")

# User input: dietary preferences
col1, col2 = st.columns(2)
with col1:
    diet = st.selectbox("Diet Type", ["Any", "Vegan", "Vegetarian", "Keto", "Paleo", "Gluten-Free"])
with col2:
    goal = st.selectbox("Health Goal", ["General Health", "Weight Loss", "Muscle Gain", "Low Carb", "Low Sugar"])

calorie_limit = st.slider("Max Calories Per Meal", min_value=100, max_value=1200, value=600, step=50)

user_query = st.text_input("Enter your question (e.g., suggest a vegan dinner under 500 calories)")

if st.button("Get Recommendation") and user_query:
    with st.spinner("Thinking..."):
        input_vars = {
            "diet": diet,
            "goal": goal,
            "calorie_limit": calorie_limit,
            "query": user_query
        }
        response = dietary_qa_chain.run(input_vars)
        st.markdown("### 🍽️ NutriBot Suggests:")
        st.markdown(response)

### Type-3: Calorie & Macronutrient Calculator integration

##### File: RAG_chain

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.embeddings import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain.chains import LLMChain, RetrievalQA
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from pinecone import Pinecone, ServerlessSpec
import os
import getpass

In [ ]:
# Load Pinecone index
vectorstore = Pinecone.from_existing_index(index_name="nutrition-rag-index", embedding=embedding)

retriever = vectorstore.as_retriever(search_type="similarity", k=4)

In [ ]:
# Prompt Template
prompt_template = PromptTemplate(
    input_variables=["diet", "goal", "calorie_limit", "query"],
    template="""
You are NutriBot, a smart nutrition assistant.

User's dietary profile:
- Diet type: {diet}
- Health goal: {goal}
- Max calories per meal: {calorie_limit}

Please answer the user's query:
"{query}"

Respond with a recipe suggestion including:
- Title
- Ingredients (1 per line)
- Instructions

Keep the recipe within dietary constraints.
"""
)

In [ ]:
llm_chain = LLMChain(llm=llm, prompt=prompt_template)
stuff_chain = StuffDocumentsChain(
    llm_chain=llm_chain,
    document_variable_name="context"
)

In [ ]:
dietary_qa_chain = RetrievalQA(
    retriever=retriever,
    combine_documents_chain=stuff_chain,
    return_source_documents=True
)

Macronutrient Calculator Part

In [ ]:
# Simulated nutrition DB (per 100g)
nutrition_lookup = {
    "chicken breast": {"calories": 165, "protein": 31, "fat": 3.6, "carbs": 0},
    "tofu": {"calories": 76, "protein": 8, "fat": 4.8, "carbs": 1.9},
    "broccoli": {"calories": 34, "protein": 2.8, "fat": 0.4, "carbs": 6.6},
    "olive oil": {"calories": 884, "protein": 0, "fat": 100, "carbs": 0},
    "rice": {"calories": 130, "protein": 2.4, "fat": 0.3, "carbs": 28},
    "salmon": {"calories": 208, "protein": 20, "fat": 13, "carbs": 0}
    # Extend this with more entries or link to real API
}

In [ ]:
def estimate_nutrition(ingredients: str):
    """
    Parse ingredient list and estimate macros using static lookup.
    """
    total = {"calories": 0, "protein": 0, "fat": 0, "carbs": 0}
    for line in ingredients.split("\n"):
        for item in nutrition_lookup:
            if item in line.lower():
                total["calories"] += nutrition_lookup[item]["calories"]
                total["protein"] += nutrition_lookup[item]["protein"]
                total["fat"] += nutrition_lookup[item]["fat"]
                total["carbs"] += nutrition_lookup[item]["carbs"]
    return total

In [ ]:
def get_dietary_answer(input_vars):
    result = dietary_qa_chain.run(input_vars)

    # Extract ingredient section from response
    match = re.search(r"(?i)ingredients:?(.+?)instructions:", result, re.DOTALL)
    ingredients = match.group(1).strip() if match else ""

    macros = estimate_nutrition(ingredients)
    return {
        "answer": result,
        "macros": macros
    }

##### File: Streamlit UI - main.py

In [ ]:
import streamlit as st
from chains.rag_chain import get_dietary_answer

In [ ]:
st.set_page_config(page_title="NutriBot - Nutrition Chat", page_icon="🥦")
st.title("🥦 NutriBot - Personalized Nutrition Chatbot")

st.markdown("Ask about food, recipes, or get suggestions based on your dietary goals!")

# Dietary Inputs
col1, col2 = st.columns(2)
with col1:
    diet = st.selectbox("Diet Type", ["Any", "Vegan", "Vegetarian", "Keto", "Paleo", "Gluten-Free"])
with col2:
    goal = st.selectbox("Health Goal", ["General Health", "Weight Loss", "Muscle Gain", "Low Carb", "Low Sugar"])

calorie_limit = st.slider("Max Calories Per Meal", min_value=100, max_value=1200, value=600, step=50)

user_query = st.text_input("Enter your question (e.g., suggest a vegan dinner under 500 calories)")

if st.button("Get Recommendation") and user_query:
    with st.spinner("NutriBot is thinking..."):
        input_vars = {
            "diet": diet,
            "goal": goal,
            "calorie_limit": calorie_limit,
            "query": user_query
        }
        response = get_dietary_answer(input_vars)

        st.markdown("### 🍽️ Recipe Suggestion:")
        st.markdown(response["answer"])

        st.markdown("### 🧮 Estimated Macronutrients (per meal):")
        st.metric("Calories", f"{response['macros']['calories']} kcal")
        st.metric("Protein", f"{response['macros']['protein']} g")
        st.metric("Fat", f"{response['macros']['fat']} g")
        st.metric("Carbs", f"{response['macros']['carbs']} g")

### Type-4: Recomendation YT video for suggestion meal from Serp API (YouTube)

##### File: RAG_chain

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.embeddings import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain.chains import LLMChain, RetrievalQA
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from pinecone import Pinecone, ServerlessSpec
from serpapi import GoogleSearch
import os
import getpass

In [ ]:
# Load Pinecone index
vectorstore = Pinecone.from_existing_index(index_name="nutrition-rag-index", embedding=embedding)

retriever = vectorstore.as_retriever(search_type="similarity", k=4)

In [ ]:
# Prompt
prompt_template = PromptTemplate(
    input_variables=["diet", "goal", "calorie_limit", "query"],
    template="""
You are NutriBot, a smart nutrition assistant.

User dietary profile:
- Diet: {diet}
- Goal: {goal}
- Max Calories: {calorie_limit}

Please answer:
"{query}"

Provide a recipe suggestion with:
- Title
- Ingredients (one per line)
- Instructions
"""
)

In [ ]:
llm_chain = LLMChain(llm=llm, prompt=prompt_template)
stuff_chain = StuffDocumentsChain(llm_chain=llm_chain, document_variable_name="context")
dietary_qa_chain = RetrievalQA(
    retriever=retriever,
    combine_documents_chain=stuff_chain,
    return_source_documents=True
)

In [ ]:
# Nutrition estimate lookup (simplified)
nutrition_lookup = {
    "tofu": {"calories": 76, "protein": 8, "fat": 4.8, "carbs": 1.9},
    "broccoli": {"calories": 34, "protein": 2.8, "fat": 0.4, "carbs": 6.6},
    "chicken breast": {"calories": 165, "protein": 31, "fat": 3.6, "carbs": 0},
    "rice": {"calories": 130, "protein": 2.4, "fat": 0.3, "carbs": 28},
    "salmon": {"calories": 208, "protein": 20, "fat": 13, "carbs": 0},
    "olive oil": {"calories": 884, "protein": 0, "fat": 100, "carbs": 0}
}

In [ ]:
def estimate_nutrition(ingredients: str):
    total = {"calories": 0, "protein": 0, "fat": 0, "carbs": 0}
    for line in ingredients.split("\n"):
        for item in nutrition_lookup:
            if item in line.lower():
                for key in total:
                    total[key] += nutrition_lookup[item][key]
    return total

In [ ]:
def fetch_youtube_videos(recipe_title: str, max_results: int = 2):
    search = GoogleSearch({
        "q": f"{recipe_title} recipe site:youtube.com",
        "api_key": os.getenv("SERPAPI_API_KEY"),
        "num": max_results,
        "engine": "youtube"
    })
    results = search.get_dict()
    return results.get("video_results", [])[:max_results]

In [ ]:
def get_dietary_answer(input_vars):
    result = dietary_qa_chain.run(input_vars)

    # Extract ingredients
    match = re.search(r"(?i)ingredients:?(.+?)instructions:", result, re.DOTALL)
    ingredients = match.group(1).strip() if match else ""

    # Extract recipe title
    title_match = re.search(r"(?i)^([^\n]+)\n", result.strip())
    recipe_title = title_match.group(1).strip() if title_match else input_vars["query"]

    # Estimate macros
    macros = estimate_nutrition(ingredients)

    # Fetch YouTube videos
    videos = fetch_youtube_videos(recipe_title)

    return {
        "answer": result,
        "macros": macros,
        "videos": videos
    }

##### File: Streamlit UI - main.py

In [ ]:
import streamlit as st
from chains.rag_chain import get_dietary_answer

In [ ]:
st.set_page_config(page_title="NutriBot - Nutrition Chat", page_icon="🥦")
st.title("🥦 NutriBot - Personalized Nutrition Chatbot")

st.markdown("Ask about food, recipes, or get suggestions based on your dietary goals!")

# User Inputs
col1, col2 = st.columns(2)
with col1:
    diet = st.selectbox("Diet Type", ["Any", "Vegan", "Vegetarian", "Keto", "Paleo", "Gluten-Free"])
with col2:
    goal = st.selectbox("Health Goal", ["General Health", "Weight Loss", "Muscle Gain", "Low Carb", "Low Sugar"])

calorie_limit = st.slider("Max Calories Per Meal", min_value=100, max_value=1200, value=600, step=50)

user_query = st.text_input("Enter your question (e.g., suggest a vegan dinner under 500 calories)")

if st.button("Get Recommendation") and user_query:
    with st.spinner("NutriBot is thinking..."):
        input_vars = {
            "diet": diet,
            "goal": goal,
            "calorie_limit": calorie_limit,
            "query": user_query
        }
        response = get_dietary_answer(input_vars)

        st.markdown("### 🍽️ Recipe Suggestion")
        st.markdown(response["answer"])

        st.markdown("### 🧮 Estimated Macronutrients")
        col1, col2, col3, col4 = st.columns(4)
        col1.metric("Calories", f"{response['macros']['calories']} kcal")
        col2.metric("Protein", f"{response['macros']['protein']} g")
        col3.metric("Fat", f"{response['macros']['fat']} g")
        col4.metric("Carbs", f"{response['macros']['carbs']} g")

        if response["videos"]:
            st.markdown("### 🎥 Cooking Videos from YouTube")
            for vid in response["videos"]:
                st.markdown(f"**[{vid['title']}]({vid['link']})**")
                st.image(vid["thumbnail"]["static"], width=320)
        else:
            st.info("No videos found for this recipe.")

### Type-5: Remember the user's dietary preferences (e.g., vegan) and past queries (e.g., “low carb dinner”).

##### File: RAG_chain

In [ ]:
import re

In [ ]:
def update_user_profile(session_state, diet, goal, calorie_limit):
    session_state["user_profile"] = {
        "diet": diet,
        "goal": goal,
        "calorie_limit": calorie_limit
    }

def save_user_history(session_state, query, result):
    history = session_state.get("query_history", [])
    history.append({"query": query, "result": result})
    session_state["query_history"] = history

def get_dietary_answer(input_vars, session_state=None):
    result = dietary_qa_chain.run(input_vars)

    # Extract ingredients
    match = re.search(r"(?i)ingredients:?(.+?)instructions:", result, re.DOTALL)
    ingredients = match.group(1).strip() if match else ""

    # Extract recipe title
    title_match = re.search(r"(?i)^([^\n]+)\n", result.strip())
    recipe_title = title_match.group(1).strip() if title_match else input_vars["query"]

    # Estimate macros
    macros = estimate_nutrition(ingredients)

    # Fetch YouTube videos
    videos = fetch_youtube_videos(recipe_title)

    # Save query + result to session history
    if session_state is not None:
        save_user_history(session_state, input_vars["query"], result)

    return {
        "answer": result,
        "macros": macros,
        "videos": videos
    }

##### File: Streamlit UI - main.py

In [ ]:
import streamlit as st
from chains.rag_chain import (
    get_dietary_answer,
    update_user_profile
)

In [ ]:
st.set_page_config(page_title="NutriBot - Nutrition Chat", page_icon="🥦")
st.title("🥦 NutriBot - Personalized Nutrition Chatbot")

st.markdown("Welcome back! NutriBot remembers your preferences and questions during this session.")

# ----------- Initialize session state ----------
if "user_profile" not in st.session_state:
    st.session_state["user_profile"] = {}

if "query_history" not in st.session_state:
    st.session_state["query_history"] = []

# ----------- User Profile Inputs ----------
st.sidebar.header("🧑‍🍳 Your Preferences")
diet = st.sidebar.selectbox("Diet Type", ["Any", "Vegan", "Vegetarian", "Keto", "Paleo", "Gluten-Free"])
goal = st.sidebar.selectbox("Health Goal", ["General Health", "Weight Loss", "Muscle Gain", "Low Carb", "Low Sugar"])
calorie_limit = st.sidebar.slider("Max Calories", 100, 1200, 600, 50)

# Update profile in session state
update_user_profile(st.session_state, diet, goal, calorie_limit)

# ----------- User Query ----------
st.subheader("🔎 Ask NutriBot a Question")
user_query = st.text_input("Example: Suggest a vegan dinner under 500 calories")

if st.button("Get Recommendation") and user_query:
    with st.spinner("NutriBot is preparing your recommendation..."):
        input_vars = {
            "diet": diet,
            "goal": goal,
            "calorie_limit": calorie_limit,
            "query": user_query
        }
        response = get_dietary_answer(input_vars, session_state=st.session_state)

        st.markdown("### 🍽️ Recipe Suggestion")
        st.markdown(response["answer"])

        st.markdown("### 🧮 Estimated Macronutrients")
        col1, col2, col3, col4 = st.columns(4)
        col1.metric("Calories", f"{response['macros']['calories']} kcal")
        col2.metric("Protein", f"{response['macros']['protein']} g")
        col3.metric("Fat", f"{response['macros']['fat']} g")
        col4.metric("Carbs", f"{response['macros']['carbs']} g")

        if response["videos"]:
            st.markdown("### 🎥 Related Cooking Videos")
            for vid in response["videos"]:
                st.markdown(f"[{vid['title']}]({vid['link']})")
                st.image(vid["thumbnail"]["static"], width=320)

# ----------- Display Query History ----------
if st.session_state["query_history"]:
    st.markdown("---")
    st.subheader("🕒 Your Recent Queries")
    for item in reversed(st.session_state["query_history"][-5:]):  # last 5
        st.markdown(f"- **Query:** {item['query']}")
        st.markdown(f"  _Recipe:_ {item['result'].splitlines()[0]}")

### Type-6: Weekly Meal Plan Builder based on user-selected favorite recipes

##### File: RAG_chain

In [ ]:
def get_dietary_answer(input_vars, session_state=None):
    result = dietary_qa_chain.run(input_vars)

    # Extract ingredients
    match = re.search(r"(?i)ingredients:?(.+?)instructions:", result, re.DOTALL)
    ingredients = match.group(1).strip() if match else ""

    # Extract recipe title
    title_match = re.search(r"(?i)^([^\n]+)\n", result.strip())
    recipe_title = title_match.group(1).strip() if title_match else input_vars["query"]

    # Estimate macros
    macros = estimate_nutrition(ingredients)

    # Fetch YouTube videos
    videos = fetch_youtube_videos(recipe_title)

    # Save query + result to session history
    if session_state is not None:
        save_user_history(session_state, input_vars["query"], result)

    return {
        "title": recipe_title,
        "answer": result,
        "macros": macros,
        "videos": videos,
        "ingredients": ingredients
    }

##### File: Streamlit UI - main.py

In [ ]:
import streamlit as st
from chains.rag_chain import get_dietary_answer, update_user_profile

st.set_page_config(page_title="NutriBot - Nutrition Chat", page_icon="🥦")

# ---------- Init State ----------
if "user_profile" not in st.session_state:
    st.session_state["user_profile"] = {}

if "query_history" not in st.session_state:
    st.session_state["query_history"] = []

if "favorites" not in st.session_state:
    st.session_state["favorites"] = []

if "weekly_plan" not in st.session_state:
    st.session_state["weekly_plan"] = {
        "Monday": "", "Tuesday": "", "Wednesday": "",
        "Thursday": "", "Friday": "", "Saturday": "", "Sunday": ""
    }

# ---------- Tabs ----------
tab1, tab2 = st.tabs(["Chat with NutriBot", "Weekly Meal Planner"])

# ---------- TAB 1: Chatbot ----------
with tab1:
    st.title("NutriBot - Personalized Nutrition Chatbot")

    st.sidebar.header("Your Preferences")
    diet = st.sidebar.selectbox("Diet Type", ["Any", "Vegan", "Vegetarian", "Keto", "Paleo", "Gluten-Free"])
    goal = st.sidebar.selectbox("Health Goal", ["General Health", "Weight Loss", "Muscle Gain", "Low Carb", "Low Sugar"])
    calorie_limit = st.sidebar.slider("Max Calories", 100, 1200, 600, 50)

    update_user_profile(st.session_state, diet, goal, calorie_limit)

    st.subheader("Ask NutriBot a Question")
    user_query = st.text_input("E.g. suggest a high-protein vegetarian lunch")

    if st.button("Get Recommendation") and user_query:
        with st.spinner("Thinking..."):
            input_vars = {
                "diet": diet,
                "goal": goal,
                "calorie_limit": calorie_limit,
                "query": user_query
            }
            response = get_dietary_answer(input_vars, session_state=st.session_state)

            st.session_state["last_response"] = response

            st.markdown("### Recipe")
            st.markdown(response["answer"])

            st.markdown("### Macronutrients")
            col1, col2, col3, col4 = st.columns(4)
            col1.metric("Calories", f"{response['macros']['calories']} kcal")
            col2.metric("Protein", f"{response['macros']['protein']} g")
            col3.metric("Fat", f"{response['macros']['fat']} g")
            col4.metric("Carbs", f"{response['macros']['carbs']} g")

            st.markdown("### Cooking Videos")
            for vid in response["videos"]:
                st.markdown(f"[{vid['title']}]({vid['link']})")
                st.image(vid["thumbnail"]["static"], width=300)

            if st.button("Save to Favorites"):
                st.session_state["favorites"].append({
                    "title": response["title"],
                    "answer": response["answer"]
                })
                st.success(f"'{response['title']}' added to favorites!")

# ---------- TAB 2: Weekly Meal Planner ----------
with tab2:
    st.header("Weekly Meal Plan Builder")

    if not st.session_state["favorites"]:
        st.info("No favorites saved yet. Go to the Chat tab and save some recipes.")
    else:
        days = list(st.session_state["weekly_plan"].keys())
        titles = [f["title"] for f in st.session_state["favorites"]]

        for day in days:
            selected = st.selectbox(f"{day}", [""] + titles, key=day)
            st.session_state["weekly_plan"][day] = selected

        if st.button("Generate Meal Plan"):
            st.subheader("Your Weekly Plan")
            for day in days:
                meal = st.session_state["weekly_plan"][day]
                if meal:
                    st.markdown(f"**{day}** : {meal}")
                else:
                    st.markdown(f"**{day}**: _No meal assigned_")

        if st.button("Clear Plan"):
            for day in days:
                st.session_state["weekly_plan"][day] = ""
            st.success("Meal plan cleared.")

### Type-7: Chat memory using ConversationBufferMemory and gemini RAG

##### File: RAG_chain

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.embeddings import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain.chains import ConversationalRetrievalChain
from pinecone import Pinecone, ServerlessSpec
import os
import getpass

In [ ]:
# Load Pinecone index
vectorstore = Pinecone.from_existing_index(index_name="nutrition-rag-index", embedding=embedding)

retriever = vectorstore.as_retriever(search_type="similarity", k=4)

In [ ]:
# Prompt Template
prompt_template = PromptTemplate(
    input_variables=["chat_history", "question"],
    template="""
You are NutriBot, a smart nutrition assistant.

Use the conversation history and context to answer user questions naturally.

Chat history:
{chat_history}

User question:
{question}

Your helpful answer:
"""
)

In [ ]:
# Memory initialization
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

# RAG with chat memory
chat_rag = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    combine_docs_chain_kwargs={"prompt": prompt_template}
)

def get_memory_rag_answer(query: str):
    return chat_rag.run(query)

def get_chat_history():
    return memory.chat_memory.messages

##### File: Streamlit UI - main.py

In [ ]:
import streamlit as st
from chains.rag_chain import get_memory_rag_answer, get_chat_history

st.set_page_config(page_title="NutriBot - Smart Chat", page_icon="🥗")

st.title("🥗 NutriBot - Nutrition Assistant with Memory")

# Session history tracker (optional for re-display)
if "chat_messages" not in st.session_state:
    st.session_state["chat_messages"] = []

# Input
query = st.chat_input("Ask something like: suggest a vegan lunch under 500 kcal")

if query:
    with st.spinner("NutriBot is thinking..."):
        response = get_memory_rag_answer(query)
        st.session_state["chat_messages"].append({"user": query, "bot": response})

# Show chat history
st.markdown("### 🧠 Chat History")
for msg in st.session_state["chat_messages"]:
    st.markdown(f"**You:** {msg['user']}")
    st.markdown(f"**NutriBot:** {msg['bot']}")

### Type-8: Create multi-tab UI (e.g. Chat, Meal Plan, Favorites) on streamlit

##### File: RAG_chain

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.embeddings import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain.chains import ConversationalRetrievalChain
from pinecone import Pinecone, ServerlessSpec
import re
import os
import getpass

In [ ]:
# Gemini LLM
llm = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.7)

# Embeddings + Pinecone setup
embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
pinecone.init(api_key=os.getenv("PINECONE_API_KEY"), environment=os.getenv("PINECONE_ENVIRONMENT"))
vectorstore = Pinecone.from_existing_index(index_name=os.getenv("PINECONE_INDEX_NAME"), embedding=embedding)
retriever = vectorstore.as_retriever(search_type="similarity", k=4)

# Memory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# Prompt Template
prompt_template = PromptTemplate(
    input_variables=["chat_history", "question"],
    template="""
You are NutriBot, a helpful and personalized nutrition assistant.

Use the chat history to understand context and provide accurate, relevant responses.

Chat history:
{chat_history}

User question:
{question}

NutriBot answer:
"""
)

# Conversational Retrieval Chain
chat_rag = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    combine_docs_chain_kwargs={"prompt": prompt_template}
)

def get_memory_rag_answer(query: str):
    return chat_rag.run(query)

def get_chat_history():
    return memory.chat_memory.messages

##### File: Streamlit UI - main.py

In [ ]:
import streamlit as st
from chains.rag_chain import get_memory_rag_answer

st.set_page_config(page_title="NutriBot - AI Nutrition Assistant", page_icon="🥦")

st.title("🥦 NutriBot - Your AI Nutrition Coach")

# Initialize session state
for key in ["chat_messages", "favorites", "weekly_plan"]:
    if key not in st.session_state:
        st.session_state[key] = []

if not st.session_state["weekly_plan"]:
    st.session_state["weekly_plan"] = {
        "Monday": "", "Tuesday": "", "Wednesday": "",
        "Thursday": "", "Friday": "", "Saturday": "", "Sunday": ""
    }

# ------------ Multi-Tab UI ------------
tab_chat, tab_meal_plan, tab_favorites = st.tabs(["🧠 Chat", "📅 Meal Plan", "❤️ Favorites"])

# ---------- TAB 1: Chat ----------
with tab_chat:
    st.subheader("🧠 Chat with NutriBot")

    query = st.chat_input("Ask me something like: 'suggest a vegan lunch under 600 kcal'")
    if query:
        with st.spinner("Thinking..."):
            response = get_memory_rag_answer(query)
            st.session_state["chat_messages"].append({"user": query, "bot": response})
            st.session_state["last_response"] = {"title": query.split()[0].capitalize(), "answer": response}

    # Chat history display
    for msg in st.session_state["chat_messages"]:
        st.markdown(f"**You:** {msg['user']}")
        st.markdown(f"**NutriBot:** {msg['bot']}")

    if "last_response" in st.session_state:
        if st.button("❤️ Save Last Recipe to Favorites"):
            st.session_state["favorites"].append(st.session_state["last_response"])
            st.success("Saved to Favorites!")

# ---------- TAB 2: Meal Plan ----------
with tab_meal_plan:
    st.subheader("📅 Weekly Meal Planner")

    if not st.session_state["favorites"]:
        st.info("You haven't saved any favorites yet. Chat with NutriBot and click '❤️ Save'.")
    else:
        recipe_titles = [f["title"] for f in st.session_state["favorites"]]
        for day in st.session_state["weekly_plan"].keys():
            st.session_state["weekly_plan"][day] = st.selectbox(
                f"{day} Meal", [""] + recipe_titles, key=f"meal_{day}"
            )

        if st.button("✅ Show Meal Plan"):
            st.markdown("### 🗓️ Your Weekly Meal Plan")
            for day, recipe in st.session_state["weekly_plan"].items():
                st.markdown(f"**{day}**: {recipe or '_No meal selected_'}")

        if st.button("🗑️ Clear Meal Plan"):
            for day in st.session_state["weekly_plan"].keys():
                st.session_state["weekly_plan"][day] = ""
            st.success("Meal plan cleared.")

# ---------- TAB 3: Favorites ----------
with tab_favorites:
    st.subheader("❤️ Your Favorite Recipes")

    if not st.session_state["favorites"]:
        st.info("No favorites saved yet.")
    else:
        for i, fav in enumerate(st.session_state["favorites"]):
            st.markdown(f"### 🍽️ {fav['title']}")
            st.markdown(fav["answer"])
            if st.button(f"🗑️ Remove", key=f"remove_{i}"):
                st.session_state["favorites"].pop(i)
                st.success("Removed.")
                st.experimental_rerun()